<h1>Interacting with the VAMPS-API</h1>
<h2>This notebook:</h2>
<ul>
<li><strong>Logs into VAMPS account </strong> if you don't have an account, use guest account (Username: 'guest', Password: 'guest')</li>
<li><strong>Uses a config file</strong></li>
Either specify an existing config file to use <strong>OR</strong> edit 'config' in the code below to match your preferences
<li><strong>Gets dataset ID's for specified dataset</strong></li>
<li><strong>Displays specified visualization</strong></li>
<li><strong>Saves visualization locally to your computer</strong></li>
<li><strong>Can upload config to VAMPS for later use</strong></li>
</ul>
<h2>To use this notebook:</h2>
<ul><strong><li>Press run cell button</li>
<li>Enter information if promtped</li>
<li>Do not run the next cell if the previous cell has an asterisk next to "In" ("In [*]:"), this means it is still processing</li>
<li>When the asterisk turns into any number, you can run the next cell</li></strong></ul>

<h3>Import relevant python packages; Allow both Python 2 and 3</h3>

In [ ]:
"""
Author: AAV / SR
vamps2.mbl.edu
"""

import os,sys
import requests
from bs4 import BeautifulSoup   # parser for html
import json, string, getpass

#to allow both Python 2 and 3
try:
    input = raw_input  
except NameError: #Python 3
    pass


<h3>Gets VAMPS username and password, then attempts login to VAMPS</h3>

In [19]:
#get VAMPS username and password
user = input("Enter your VAMPS username: ")
pw = getpass.getpass("Enter your VAMPS password: ")

conn = {'user': user,
        'passwd': pw,
         # vamps:             https://vamps2.mbl.edu
         # vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
         # localhost:         http://localhost:3000 
        'hosturl':'https://vamps2.mbl.edu'
       }

#attempt login to VAMPS
s = requests.Session()
r = s.post(conn['hosturl']+'/users/login', data={'username':conn['user'], 'password':conn['passwd']})

Enter your VAMPS username: ruzics
Enter your VAMPS password: ········


<h3>If username/password is incorrect, exit program</h3>

In [20]:
#exit program if login unsuccessful
if r.url == 'https://vamps2.mbl.edu/users/login':
    sys.exit('Login not successful')
elif r.url == 'https://vamps2.mbl.edu/':
    print('Login successful')

Login successful


<h3>Option to upload an existing config file or see list of datasets</h3>

In [22]:
upload = input("Do you want to use an already existing config file? ('Y' or 'N'): ")

#to upload config: 
if upload[0].capitalize() == "Y":
    file = input('Enter JSON Config File: ')
    with open(file) as f:        
        config = json.load(f)
    id_list = 'N'
else:
    id_list = input("Do you want to search through datasets or see all you have access to? ('Y' or 'N'): ")
    print("Edit 'config' below to match preferences before running cell")

Do you want to use an already existing config file? ('Y' or 'N'): Y
Enter JSON Config File: test_api.json


<h3>Option to search through datasets you have access to or see all</h3>

In [23]:
if id_list[0].capitalize() == "Y":
    
    #use selenium since requests cannot process Javascript on page
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    
    #open remote browser
    browser = webdriver.PhantomJS()
    browser.get('https://vamps2.mbl.edu/users/login')
    
    #log into VAMPS
    browser.find_element_by_id("username").send_keys(user)
    browser.find_element_by_id("password").send_keys(pw)
    browser.find_element_by_id("password").submit();
    
    #get ID selection page
    browser.get('https://vamps2.mbl.edu/visuals/visuals_index')
    
    #wait for ID sets to load
    timeout = 30
    element_present = EC.presence_of_element_located((By.ID, "project/AB_SAND_Bv6/Microbial populations in sand/public"))
    WebDriverWait(browser, timeout).until(element_present)
    #print(browser.page_source)

    #make a list of all dataset objects
    elements = browser.find_elements_by_class_name("tooltip_pjds_list")
    
    search = input("Enter dataset you are looking for to get a list of matches, or enter 'all' to see all datasets: ")
    search = search.upper()
    if search == 'ALL':
        print("Here is a list of datasets you have access to: ")
        for element in elements:
            print(element.text)
    else:
        print("Here are your search results: ")
        print("You can use any of these in the 'project' field of config below")
        for element in elements:
            if search in element.text:
                print(element.text)
        
    

Enter dataset you are looking for to get a list of matches, or enter 'all' to see all datasets: No
Here are your search results: 
You can use any of these in the 'project' field of config below


<h3>If no config file was uploaded, edit 'config' *before* you run the cell. The comments show the options for each value.</h3>

In [24]:
if upload == "N":
    #edit config, comments next to values show allowed different options
    config = {
        "api":"1",
        "source":"VAMPS-API",
        "update_data":1,
        "normalization":"none",              # none, maximum, frequency
        "selected_distance":"morisita-horn",  # morisita-horn, jaccard, kulczynski, canberra bray-curtis
        "tax_depth":"phylum",                 # domain, phylum, klass, order, family, genus, species, strain
        "domains":["Archaea","Bacteria","Eukarya","Organelle","Unknown"],   #["Archaea","Bacteria","Eukarya","Organelle","Unknown"],
        "include_nas":"yes",                  # yes or no
        "min_range":0,                        # integer 0-99
        "max_range":100,                      # integer 1-100

          # Must be a valid project - with correct permissions for the above user.
        'project':'MVCO_ciliate_timeseries2',   

          # Currently avalable: "dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "fheatmap", "dendrogram" 
        'image':'adiversity'
    }

<h3>Get and display project IDs</h3>

In [25]:
# get project ids:
r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.text
print(config['ds_order'])

[353061,353062,353063,353064,353065,353066,353067,353068,353069,353070,353071,353072,353073,353074,353075,353076,353077,353078,353079,353080,353081,353082,353083,353084,353085,353086,353087]


<h3>Create remote configuration and get timestamp (file_prefix)</h3>

In [26]:
# Get timestamp (filename prefix):
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string

<h3>Save matrix file which is integral to VAMPS images</h3>

In [27]:
import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)

<h3>Save image file</h3>

In [28]:
r = s.post(conn['hosturl']+'/api/create_image', timeout=30, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
print('Done writing local file:',local_filename)

ruzics_1501604711304-adiversity-api.csv
Done writing local file: ruzics_1501604711304-adiversity-api.csv


<h3>Option to upload config file to VAMPS for later use</h3>

In [29]:
upload_to_vamps = input("Do you want to upload config file to VAMPS for later use?('Y' or 'N'): ")
if upload_to_vamps[0].capitalize() == 'Y':
    dataset_ids = config['ds_order'].strip('[')
    dataset_ids = dataset_ids.strip(']')
    dataset_ids = dataset_ids.split(',')
    vamps_config = {
      "source":"VAMPS",
      "image":config['image'],
      "post_items":
      { 
        "normalization":config['normalization'],
        "selected_distance":config['selected_distance'],
        "tax_depth":config['tax_depth'],
        "domains":config['domains'],
        "include_nas":config['include_nas'],
        "min_range":config['min_range'],
        "max_range":config['max_range']
      },
        "id_name_hash":
        {
          "ids":dataset_ids
        }
    }
    upload = {
    'upload_files': json.dumps(vamps_config)
    }
    data = {
    'from_upload_configuration_file' : '1'
    }
    r = s.post('https://vamps2.mbl.edu/visuals/view_selection', data=data,files=upload)

Do you want to upload config file to VAMPS for later use?('Y' or 'N'): N


<h3>Show image (if chosen image is implemented)</h3>

In [30]:
ready_images = ["dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "fheatmap", "dendrogram" ]

from IPython.core.display import display, HTML
out = ''
if config['image'] in ready_images:
    if config['image'] == 'metadata_csv':
        print(return_result)
    else:
        out = HTML("<style>.container { width:100% !important; }</style>"+return_result)
else:
    out = 'not implemented yet'
out

Dataset,observed richness,ACE,chao1,Shannon,Simpson
MVCO_ciliate_timeseries--RG10join,3,4.11111111111,3.0,0.0170213665167,0.00297609500596
MVCO_ciliate_timeseries--RG11join,7,17.0,10.0,0.325338004629,0.0990887949372
MVCO_ciliate_timeseries--RG12join,7,10.0277777778,7.5,0.271760430211,0.0695136798539
MVCO_ciliate_timeseries--RG13join,19,22.7195216049,22.3333333333,1.06850976048,0.302325350072
MVCO_ciliate_timeseries--RG14join,2,error,2.0,0.00436210061712,0.000671817189849
MVCO_ciliate_timeseries--RG15join,6,7.25850340136,7.0,0.0553278346642,0.00990050941675
MVCO_ciliate_timeseries--RG16join,5,11.0,6.5,0.0171704147005,0.00265589949129
MVCO_ciliate_timeseries--RG17join,7,13.5555555556,8.5,0.0917106870253,0.0180297532635
MVCO_ciliate_timeseries--RG18join,7,18.012345679,10.0,0.103464240215,0.0203961176873
MVCO_ciliate_timeseries--RG19join,4,7.9,5.0,0.041298983,0.0076897786824
